In [1]:
import MA4Star_Strategy as MA4
import os
import mplfinance as mpf
import matplotlib.pyplot as plt

2025-07-08 23:46:13.207890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-08 23:46:13.219157: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-08 23:46:13.222647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-08 23:46:13.231111: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
ticker = "SPY"
window_days = 55
alpha = 0.5
star_lbdays=0
predict_days=5
data_label = MA4.data_label(ticker, window_days, alpha, predict_days, star_lbdays)
print(data_label)

SPY_55_0.5_5_0


In [3]:
batch_size = 32
epochs = 30
model_label = MA4.model_label(data_label, "CNN", epochs, batch_size)
print(model_label)
model = MA4.retreive_model(model_label)
if model is not None:
    model.summary()

SPY_55_0.5_5_0_CNN_30_32


I0000 00:00:1752018374.907029     605 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752018374.916575     605 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752018374.916782     605 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1752018374.918477     605 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model SPY_55_0.5_5_0_CNN_30_32 loaded successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,723,339 (273.60 MB)

 Trainable params: 23,907,779 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 47,815,560 (182.40 MB)

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [5]:
period=f"{window_days+1}d"
print(period)
enddate = date(2025, 7, 3)
startdate = enddate - timedelta(days=window_days+1)
print(startdate, ' , ', enddate)
data = yf.download(ticker, start=startdate, end=enddate, progress=False)

56d
2025-05-08  ,  2025-07-03


/tmp/ipykernel_605/632347916.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=startdate, end=enddate, progress=False)


In [6]:
data.columns = [col[0] for col in data.columns]
display(data.shape)
display(data.head(2))
display(data.tail(2))

(38, 5)

,Close,High,Low,Open,Volume
Date,,,,,
2025-05-08,563.394470,568.628996,560.044388,563.573932,65130800
2025-05-09,562.676575,565.827233,561.101215,564.810221,37603400


,Close,High,Low,Open,Volume
Date,,,,,
2025-07-01,617.650024,618.830017,615.520020,616.359985,70030100
2025-07-02,620.450012,620.489990,616.609985,617.239990,66510400


In [7]:
# Calculate EMAs
data = MA4.calculate_emas(data)
data['EMStrend'] = None
data['Labels'] = None
data['RisingStar'] = None
data['EveningStar'] = None
# Convert to numeric type with np.nan for None
data['RisingStar'] = pd.to_numeric(data['RisingStar'], errors='coerce')
data['EveningStar'] = pd.to_numeric(data['EveningStar'], errors='coerce')
data = MA4.calulate_Alltrend(data, 0, alpha) 
data.head()

,Close,High,Low,Open,Volume,EMA_5,EMA_10,EMA_20,EMA_30,EMStrend,Labels,RisingStar,EveningStar
Date,,,,,,,,,,,,,
2025-05-08,563.394470,568.628996,560.044388,563.573932,65130800,563.394470,563.394470,563.394470,563.394470,None,None,NaN,NaN
2025-05-09,562.676575,565.827233,561.101215,564.810221,37603400,563.155172,563.263944,563.326099,563.348154,SIDEWAY,None,NaN,NaN
2025-05-12,581.271606,581.281587,575.339132,579.756067,78993600,569.193983,566.538064,565.035195,564.504506,DOWN,None,NaN,NaN
2025-05-13,585.110291,587.343678,581.122081,581.690347,67947200,574.499419,569.914833,566.947109,565.833912,UP,None,NaN,NaN
2025-05-14,585.858093,587.243950,583.814087,586.077416,66283500,578.285644,572.813607,568.748155,567.125794,UP,None,NaN,NaN


In [8]:
# find Rising Star and Evening Star patterns
data = MA4.find_RE_star(data, 0, star_lbdays)
display(data.head())

Rising Star markers: 0 non-NaN values
Evening Star markers: 0 non-NaN values


,Close,High,Low,Open,Volume,EMA_5,EMA_10,EMA_20,EMA_30,EMStrend,Labels,RisingStar,EveningStar
Date,,,,,,,,,,,,,
2025-05-08,563.394470,568.628996,560.044388,563.573932,65130800,563.394470,563.394470,563.394470,563.394470,None,None,NaN,NaN
2025-05-09,562.676575,565.827233,561.101215,564.810221,37603400,563.155172,563.263944,563.326099,563.348154,SIDEWAY,None,NaN,NaN
2025-05-12,581.271606,581.281587,575.339132,579.756067,78993600,569.193983,566.538064,565.035195,564.504506,DOWN,None,NaN,NaN
2025-05-13,585.110291,587.343678,581.122081,581.690347,67947200,574.499419,569.914833,566.947109,565.833912,UP,None,NaN,NaN
2025-05-14,585.858093,587.243950,583.814087,586.077416,66283500,578.285644,572.813607,568.748155,567.125794,UP,None,NaN,NaN


In [9]:
display(len(data))
chart_data = data.iloc[len(data)-window_days:]
print(chart_data.index[0], " -- ", chart_data.index[-1], ' length=', len(chart_data))
# Detect Rising Star and Evening Star patterns
rising_star_markers = chart_data['RisingStar']
evening_star_markers = chart_data['EveningStar']
# Build additional plots list, only include scatter plots if markers exist
ap = [
    mpf.make_addplot(chart_data['EMA_5'], color='red', width=1),
    mpf.make_addplot(chart_data['EMA_10'], color='blue', width=1),
    mpf.make_addplot(chart_data['EMA_20'], color='green', width=1),
    mpf.make_addplot(chart_data['EMA_30'], color='yellow', width=1)
]
# Only add scatter plots if there are non-NaN markers
if rising_star_markers.notna().any():
    # Ensure the series contains only numeric or np.nan values
    rising_star_markers = pd.to_numeric(rising_star_markers, errors='coerce')
    print(f"Adding Rising Star markers for window {i} is ")
    ap.append(mpf.make_addplot(rising_star_markers, type='scatter', marker='^', color='cyan', markersize=100))
if evening_star_markers.notna().any():
    evening_star_markers = pd.to_numeric(evening_star_markers, errors='coerce')
    print(f"Adding Evening Star markers for window {i} is ")
    ap.append(mpf.make_addplot(evening_star_markers, type='scatter', marker='v', color='magenta', markersize=100))
start_d = chart_data.index[0].strftime('%Y%m%d')
end_d = chart_data.index[-1].strftime('%Y%m%d')
chart_path = os.path.join("charts", f"chart_{ticker}_{start_d}_{end_d}.png")
print(chart_path)
try:
    fig, ax = mpf.plot(chart_data, type='candle', addplot=ap, savefig=chart_path, warn_too_much_data=1000, returnfig=True)
    plt.close(fig)  # Close the figure to free memory
    plt.close('all')  # Close all figures to prevent memory accumulation
except Exception as e:
    print(f"Error generating chart {chart_path}: {e}")


38

2025-06-09 00:00:00  --  2025-07-02 00:00:00  length= 17
charts/chart_SPY_20250609_20250702.png


In [10]:
img = MA4.preprocess_image(chart_path)
print(img.shape)
X_test = np.array([img])
print(X_test.shape)

(224, 224, 3)
(1, 224, 224, 3)


In [11]:
y_pred = model.predict(X_test)
print(y_pred)
y_pred_classes = np.argmax(y_pred, axis=1)
print(y_pred_classes)

I0000 00:00:1752018378.340740     683 service.cc:146] XLA service 0x75cf00005210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752018378.340761     683 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-07-08 23:46:18.345080: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-08 23:46:18.372879: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
[[0.80551994 0.1883646  0.0061155 ]]
[0]


I0000 00:00:1752018378.648915     683 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [12]:
print(y_pred_classes)
print(MA4.LABEL_L[y_pred_classes[0]])

[0]
UP
